In [ ]:
import pickle
with open('E:\PaperCode\对标GraphBind\Datasets\PMG\PDB_DF\\6HMZ_X.csv.pkl', 'rb') as file:
    loaded_data = pickle.load(file)
    print(loaded_data)

In [ ]:
print(loaded_data['pdb_DF']['is_sidechain'])

In [ ]:
import numpy as np

# 假设有两个点的坐标
pointz = np.array([ 47.66925748 , 82.32201651 ,121.98438082])
point1 = np.array([ 47.72064049,  86.20137922, 122.56441245])-pointz  # 第一个点的坐标
point2 = np.array([ 51.50852779 , 87.40417973, 119.62607287])-pointz  # 第二个点的坐标
print(point1)
# 计算两个点的向量
vector1 = point1
vector2 = point2

# 计算点积
dot_product = np.dot(vector1, vector2)

# 计算向量的模长
magnitude1 = np.linalg.norm(vector1)
magnitude2 = np.linalg.norm(vector2)

# 计算夹角的余弦值
cosine_theta = dot_product / (magnitude1 * magnitude2)

# 计算夹角（弧度）
theta_rad = np.arccos(cosine_theta)

# 将弧度转换为度
theta_deg = np.degrees(theta_rad)

print("夹角 θ（度）:", theta_deg)


In [ ]:
np.sqrt(np.sum(np.square(point2)))

In [ ]:
import pandas as pd
path = 'E:/OwnCode/PionicNet/data/fasta/ionic/re/'
ionic_list = ['ZN', 'MG', 'CA', 'MN', 'FE', 'CU', 'FE2', 'NA', 'K', 'NI', 'G']
df = pd.DataFrame(columns=['ionic','number'])
for ionic in ionic_list:
    with open(path+ionic+'_label.fa','r') as file:
        df.loc[len(df)] = [ionic,int(len(file.readlines())/2)]
df

In [18]:
# 统计长度数据
import pandas as pd
path = 'E:/OwnCode/PionicNet/data/fasta/ionic/re/'
ionic_list = ['ZN', 'MG', 'CA', 'MN', 'FE', 'CU', 'FE2', 'NA', 'K', 'NI', 'G']
df = pd.DataFrame(columns=['ionic','number','0-50','50-1000','>1000'])
for ionic in ionic_list:
    with open(path+ionic+'_label.fa','r') as file:
        lines = file.readlines()
        number = int(len(lines)/2)
        lt50 = 0
        in51 = 0
        gt10 = 0
        for line in lines:
            if not line.startswith('>'):
                if len(line)-1<=50:
                    lt50 += 1
                elif len(line)-1>=1000:
                    gt10 +=1
                elif len(line)-1<1000 and len(line)-1>50:
                    in51 +=1
        df.loc[len(df)] = [ionic,number,lt50,in51,gt10]
df

,ionic,number,0-50,50-1000,>1000
0,ZN,44608,1638,40804,2166
1,MG,36017,305,34100,1612
2,CA,24358,141,23596,621
3,MN,8539,22,8363,154
4,FE,5353,3,5342,8
5,CU,3804,1,3787,16
6,FE2,2815,0,2813,2
7,NA,660,0,631,29
8,K,369,0,369,0
9,NI,328,0,328,0


In [9]:

import os

def get_all_subdirectories(directory):
    if  '.git' in directory:
        return []
    subdirectories = [f.path for f in os.scandir(directory) if f.is_dir()]
    for subdir in subdirectories:
        subdirectories.extend(get_all_subdirectories(subdir))
    return subdirectories

# 指定要获取子文件夹的目录路径
directory_path = 'J:/OwnCode/PionicNet'

# 获取所有子文件夹的列表
all_subdirectories = get_all_subdirectories(directory_path)

# 打印子文件夹列表
for subdir in all_subdirectories:
    print(subdir)



J:/OwnCode/PionicNet\.git
J:/OwnCode/PionicNet\.idea
J:/OwnCode/PionicNet\code
J:/OwnCode/PionicNet\.idea\inspectionProfiles
J:/OwnCode/PionicNet\code\data
J:/OwnCode/PionicNet\code\model
J:/OwnCode/PionicNet\code\temp
J:/OwnCode/PionicNet\code\model\__pycache__
J:/OwnCode/PionicNet\code\model\__pycache__


In [8]:
import os
import torch
from transformers import T5EncoderModel, T5Tokenizer, AutoTokenizer
import re
import numpy as np
import gc
# tokenizer = AutoTokenizer.from_pretrained("Rostlab/prot_t5_xl_uniref50", do_lower_case=False)
# model = T5EncoderModel.from_pretrained("Rostlab/prot_t5_xl_uniref50")
# Load model directly
if os.name == 'nt':
    prot_path = 'G:\Tools\prot_t5_xl_uniref50'
else:
    prot_path = ''
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
# Load the tokenizer
tokenizer = T5Tokenizer.from_pretrained(prot_path, do_lower_case=False)

# Load the model
model = T5EncoderModel.from_pretrained(prot_path).to(device)

# prepare your protein sequences as a list
sequence_examples = ["PRTEINO", "SEQWENCE"]

# replace all rare/ambiguous amino acids by X and introduce white-space between all amino acids
sequence_examples = [" ".join(list(re.sub(r"[UZOB]", "X", sequence))) for sequence in sequence_examples]

# tokenize sequences and pad up to the longest sequence in the batch
ids = tokenizer(sequence_examples, add_special_tokens=True, padding="longest")

input_ids = torch.tensor(ids['input_ids']).to(device)
attention_mask = torch.tensor(ids['attention_mask']).to(device)

# generate embeddings
with torch.no_grad():
    embedding_repr = model(input_ids=input_ids, attention_mask=attention_mask)
embedding_repr
# # extract residue embeddings for the first ([0,:]) sequence in the batch and remove padded & special tokens ([0,:7]) 
# emb_0 = embedding_repr.last_hidden_state[0,:7] # shape (7 x 1024)
# # same for the second ([1,:]) sequence but taking into account different sequence lengths ([1,:8])
# emb_1 = embedding_repr.last_hidden_state[1,:8] # shape (8 x 1024)

# # if you want to derive a single representation (per-protein embedding) for the whole protein
# emb_0_per_protein = emb_0.mean(dim=0) # shape (1024)

d:\Tools\Anaconda3\envs\onedot\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. If you see this, DO NOT PANIC! This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=True`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


KeyboardInterrupt: 

In [7]:
embedding_repr.last_hidden_state[0,:4]

NameError: name 'embedding_repr' is not defined

In [1]:
def read_fasta( fasta_path ):
    '''
        Reads in fasta file containing multiple sequences.
        Returns dictionary of holding multiple sequences or only single 
        sequence, depending on input file.
    '''
    
    sequences = dict()
    with open( fasta_path, 'r' ) as fasta_f:
        for line in fasta_f:
            # get uniprot ID from header and create new entry
            if line.startswith('>'):
                uniprot_id = line.replace('>', '').strip()
                # replace tokens that are mis-interpreted when loading h5
                uniprot_id = uniprot_id.replace("/","_").replace(".","_")
                sequences[ uniprot_id ] = ''
            else:
                # repl. all whie-space chars and join seqs spanning multiple lines
                sequences[ uniprot_id ] += ''.join( line.split() ).upper().replace("-","") # drop gaps and cast to upper-case
                
    return sequences
read_fasta('E:/OwnCode/PionicNet/data/fasta/ionic/nr/CA.fa')



{'1a0jA': 'IVGGYECRKNSASYQASLQSGYHFCGGSLISSTWVVSAAHCYKSRIQVRLGEHNIAVNEGTEQFIDSVKVIMHPSYNSRNLDNDIMLIKLSKPASLNSYVSTVALPSSCASSGTRCLVSGWGNLSGSSSNYPDTLRCLDLPILSSSSCNSAYPGQITSNMFCAGFMEGGKDSCQGDSGGPVVCNGQLQGVVSWGYGCAQRNKPGVYTKVCNYRSWISSTMSSN',
 '1a0sP': 'SGFEFHGYARSGVIMNDSGASTKSGAYITPAGETGGAIGRLGNQADTYVEMNLEHKQTLDNGATTRFKVMVADGQTSYNDWTASTSDLNVRQAFVELGNLPTFAGPFKGSTLWAGKRFDRDNFDIHWIDSDVVFLAGTGGGIYDVKWNDGLRSNFSLYGRNFGDIDDSSNSVQNYILTMNHFAGPLQMMVSGLRAKDNDERKDSNGNLAKGDAANTGVHALLGLHNDSFYGLRDGSSKTALLYGHGLGAEVKGIGSDGALRPGADTWRIASYGTTPLSENWSVAPAMLAQRSKDRYADGDSYQWATFNLRLIQAINQNFALAYEGSYQYMDLKPEGYNDRQAVNGSFYKLTFAPTFKVGSIGDFFSRPEIRFYTSWMDWSKKLNNYASDDALGSDGFNSGGEWSFGVQMETWF',
 '1a47A': 'ASDTAVSNVVNYSTDVIYQIVTDRFVDGNTSNNPTGDLYDPTHTSLKKYFGGDWQGIINKINDGYLTGMGVTAIWISQPVENIYAVLPDSTFGGSTSYHGYWARDFKRTNPYFGSFTDFQNLINTAHAHNIKVIIDFAPNHTSPASETDPTYAENGRLYDNGTLLGGYTNDTNGYFHHYGGTDFSSYEDGIYRNLFDLADLNQQNSTIDSYLKSAIKVWLDMGIDGIRLDAVKHMPFGWQKNFMDSILSYRPVFTFGEWFLGTNEIDVNNTYFANESGMSLLDFRFSQKVRQVFRDNTDTMYGLDSMIQSTASDYNFINDMVTFI

In [3]:
from pyfastx import Fasta

dic = {record.name:record.seq for record in Fasta('E:/OwnCode/PionicNet/data/fasta/ionic/nr/CA.fa')}
for (k,v) in dic.items():
    print(k,v)
    break

1a0jA IVGGYECRKNSASYQASLQSGYHFCGGSLISSTWVVSAAHCYKSRIQVRLGEHNIAVNEGTEQFIDSVKVIMHPSYNSRNLDNDIMLIKLSKPASLNSYVSTVALPSSCASSGTRCLVSGWGNLSGSSSNYPDTLRCLDLPILSSSSCNSAYPGQITSNMFCAGFMEGGKDSCQGDSGGPVVCNGQLQGVVSWGYGCAQRNKPGVYTKVCNYRSWISSTMSSN


In [1]:
import torch 
from torch import Tensor
dictT = {'a':1,'b':Tensor([1,2,3])}
print(dictT['b'])
for idx, (k,v) in enumerate(dictT.items()):
    print(idx,k,v)

tensor([1., 2., 3.])
0 a 1
1 b tensor([1., 2., 3.])
